In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs


In [2]:
from keras.applications import VGG16

size = 224

vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=(size, size, 3))

Using TensorFlow backend.


In [3]:
for layer in vgg16.layers:
    layer.trainable = False

In [4]:
# Let's print our layers 
for (i,layer) in enumerate(vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [5]:
from keras.layers import Flatten, Dropout, Dense
from keras.models import Model

adjusted_model = Flatten()(vgg16.output)
adjusted_model = Dense(256, activation="relu")(adjusted_model)
adjusted_model = Dropout(rate=0.3)(adjusted_model)
# 17 classes of flowers
adjusted_model = Dense(17, activation="softmax")(adjusted_model)

model = Model(inputs=vgg16.input, outputs=adjusted_model)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = "./data/flowers/train"
test_dir = "./data/flowers/validation"

train_data = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

test_data = ImageDataGenerator(rescale=1./255)

batch_size = 16

train_generator = train_data.flow_from_directory(
    train_dir,
    target_size=(size, size),
    class_mode="categorical",
    batch_size=batch_size,
)

test_generator = test_data.flow_from_directory(
    test_dir, 
    target_size=(size, size),
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=False
)

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


In [7]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint(
    "./flowers_v1.h5",
    monitor="val_loss",
    mode="min",
    verbose=1,
    save_best_only=True)

earlystop = EarlyStopping(
    monitor="val_loss", 
    min_delta=0,
    patience=3,
    verbose=1,
    restore_best_weights=True)

callbacks = [checkpoint, earlystop]

model.compile(loss="categorical_crossentropy", optimizer=RMSprop(lr=0.001), metrics=['accuracy'])

nb_train = 1190
nb_test = 170

epochs = 6

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train//batch_size,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1,
    validation_data=test_generator,
    validation_steps=nb_test//batch_size)

Epoch 1/6
74/74 [==============================] - 17s 232ms/step - loss: 3.8788 - accuracy: 0.2189 - val_loss: 2.0769 - val_accuracy: 0.6000

Epoch 00001: val_loss improved from inf to 2.07694, saving model to ./flowers_v1.h5
Epoch 2/6
74/74 [==============================] - 16s 218ms/step - loss: 1.9216 - accuracy: 0.4131 - val_loss: 0.3002 - val_accuracy: 0.7273

Epoch 00002: val_loss improved from 2.07694 to 0.30023, saving model to ./flowers_v1.h5
Epoch 3/6
74/74 [==============================] - 16s 212ms/step - loss: 1.5915 - accuracy: 0.4974 - val_loss: 0.3007 - val_accuracy: 0.6623

Epoch 00003: val_loss did not improve from 0.30023
Epoch 4/6
74/74 [==============================] - 15s 198ms/step - loss: 1.3140 - accuracy: 0.5656 - val_loss: 1.3923 - val_accuracy: 0.7662

Epoch 00004: val_loss did not improve from 0.30023
Epoch 5/6
74/74 [==============================] - 15s 207ms/step - loss: 1.1943 - accuracy: 0.6320 - val_loss: 0.0814 - val_accuracy: 0.8506

Epoch 00005